# Generative Agents in LangChain

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

In [1]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from termcolor import colored

In [3]:
USER_NAME = "Pam"  # The name you want to use when interviewing the agent.

# Local LLM
LLM = ChatOllama(
    model="phi3",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0,
    max_new_tokens=4096 # officailly 4096
    )

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriever, which fetches documents based on salience, recency, and importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

In [5]:
import numpy as np

import faiss


def score_normalizer(val: float) -> float:
    ret = 1.0 - 1.0 / (1.0 + np.exp(val))
    #print("val: "+str(float(val))+"_"+"ret: "+str(ret))
    return ret

def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = OllamaEmbeddings(model="phi3")

    # Automatically determine the size of the embeddings
    test_embedding = embeddings_model.embed_query("test query")
    embedding_size = len(test_embedding)
    
    # Initialize the vectorstore as empty
    index = faiss.IndexFlatL2(embedding_size)
    
    # Initialize FAISS vector store
    vectorstore = FAISS(
        embeddings_model,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=score_normalizer,
        normalize_L2=True
    )
    
    # Create and return the retriever
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, 
        other_score_keys=["importance"], 
        k=15
    )

In [6]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    memory=tommies_memory,
)

In [7]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is characterized by a strong sense of responsibility, empathy towards others, and resilience in facing challenges. They are dedicated to their commitments and strive for fairness and justice. Tommie also demonstrates adaptability and open-mindedness when encountering new situations or perspectives.


In [8]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]

for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [9]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is a reflective and empathetic individual with a strong sense of nostalgia, as evidenced by his fond memories of his childhood dog, Bruno. He appears to be adaptable and open-minded towards new experiences, such as moving into a new home and meeting the neighbors' cat. Tommie is also practical and focused on self-care, trying to rest despite external disturbances like noisy roads at night. His physical state suggests he values personal well-being, shown by his acknowledgment of fatigue from travel and hunger after a long journey.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [10]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [11]:
interview_agent(tommie, "What are you looking forward to doing today?")

/Users/motif/anaconda3/envs/langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Tommie said "Hey Pam, I\'m actually looking forward to a quiet day today. Maybe catching up on some design work and maybe exploring this new neighborhood a bit. How about you? Any plans that excite you?"'

In [12]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Well, Pam, I\'m not too worried today. It\'s been a bit of an exhausting journey getting here, but nothing that can\'t be managed with some good rest and maybe a light meal later on. The main thing is to settle in comfortably and start making this place feel like home."'

## Step through the day's observations.

In [13]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [14]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

KeyboardInterrupt: 

## Interview after the day

In [ ]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a rollercoaster today. I went to a job fair this morning and it didn\'t quite go as planned, but I\'m staying positive about my search in the design world. How has your day been?"'

In [ ]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I\'m glad we could catch up! Coffee sounds great; it always helps me refocus and energize for my job search. What about you? How do you usually start your day?"'

In [ ]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Oh, Bruno was a great companion growing up. He had this incredible energy and always seemed to sense when I needed some extra love or comfort. It\'s funny how dogs can pick up on our emotions so well. Do you have any pets that hold a special place in your heart?"'

## Adding Multiple Characters

Let's add a second character to have a conversation with Tommie. Feel free to configure different traits.

In [ ]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [ ]:
from datetime import datetime, timedelta

yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [ ]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is depicted as an early riser and diligent professional who values teamwork and camaraderie. She maintains a balanced lifestyle, engaging in recreational activities like tennis to unwind before her workday begins. Eve demonstrates empathy by overhearing concerns about colleagues' interpersonal dynamics and actively assists coworkers when needed. Her choice of breakfast suggests an appreciation for simple, nutritious meals.


## Pre-conversation interviews


Let's "Interview" Eve before she speaks with Tommie.

In [ ]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling quite prepared for today; a good night\'s rest and my morning routine always set me up well. How about you? Any particular plans or concerns that I can help with?"'

In [ ]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I haven\'t worked directly with Tommie extensively, but I\'ve heard a few colleagues mention some challenges they face when collaborating. If there are specific issues you\'re concerned about or if we can find ways to improve teamwork overall, I\'d be more than willing to help facilitate that."'

In [ ]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "It sounds like Tommie is actively seeking employment. If you\'re looking for advice or support in your job search, I can share some general tips and resources that might be helpful. However, please remember my insights are based on what others have shared with me rather than personal experience."'

In [ ]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "Absolutely, I understand that navigating a new work environment can be challenging. If you\'d like, we could schedule some time later today or another day where we can discuss this further and perhaps brainstorm ways to make the transition smoother for everyone involved."'

## Dialogue between Generative Agents

Generative agents are much more complex when they interact with a virtual environment or with each other. Below, we run a simple conversation between Tommie and Eve.

In [ ]:
from typing import List

def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [ ]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

Eve said "Hello Tommie, I appreciate your willingness to seek guidance. Let's start by discussing what you hope to achieve from our conversation today."
Tommie said "Absolutely, Eve. I'm looking forward to gaining insights that can help me navigate my job search more effectively and find a role where I can truly thrive."
Eve said "It's great that you're open to learning from different perspectives. Let's dive into your goals and see how we can align them with potential opportunities out there."
Tommie said "Absolutely, Eve. I'm all about growth and learning new things in this industry. Let's explore those opportunities together!"
Eve said "That's a wonderful attitude, Tommie. Let's outline some key areas where you can focus your energy and discuss how we might align those with potential job opportunities that value continuous learning and growth."
Tommie said "I'm really grateful for your guidance, Eve. Outlining key areas of focus sounds like a solid plan. I'm particularly interested 

## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

In [ ]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is an enthusiastic individual with a strong sense of empathy, as seen in his interactions about childhood memories and appreciation for pets that provide emotional support. He values guidance from mentors like Eve, who help him articulate his career aspirations and personal values within the design industry. Tommie is proactive in seeking opportunities to advance his career, such as attending job fairs and applying for positions, while maintaining a positive outlook even when faced with setbacks. He demonstrates gratitude towards those who support him on his journey and eagerly anticipates future sessions that will further shape his professional narrative.


In [ ]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve and Tommie, both professionals in the design industry, are preparing for a collaborative session focused on crafting Tommie's career narrative. Eve expresses her enthusiasm about working with Tommie and acknowledges his dedication to developing an inspiring story that reflects his passion for design. She emphasizes their upcoming meeting on Thursday, where they will work closely together to refine the portfolio pieces and create a compelling narrative.

Tommie reciprocates Eve's support by expressing gratitude for her guidance and excitement about the session. He looks forward to bringing his full commitment and openness to shape an impactful career story that resonates with both personal values and professional goals. Tommie is eager to meet on Thursday, ready to collaborate closely with Eve in crafting a narrative that showcases his journey as a designer.


In [ ]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It was incredibly productive and inspiring! Eve always knows how to guide me towards my goals. I\'m really looking forward to our session on Thursday where we can dive even deeper into crafting a narrative that truly represents who I am as a designer."'

In [ ]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was truly inspiring! We\'re both excited about our upcoming session and can\'t wait to dive into creating a narrative that not only showcases Tommie\'s exceptional talents but also resonates with his core values in the design world. It feels like we\'re on the brink of something truly transformative for him."'

In [ ]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "I\'m glad we had this chance to align our goals for Tommie\'s career narrative. It\'s essential that he feels heard and supported throughout this process. I believe by focusing on his strengths, passions, and the unique experiences from his time in advertising, we can craft a story that not only showcases his talents but also inspires others who might be considering a similar transition. Let\'s ensure our next session is as productive as possible for Tommie."'